<a href="https://colab.research.google.com/github/karchx/llm_scratch/blob/main/scratch/with_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>